In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Risk_Score_Predication_With_ML_FLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Risk_Score_Predication_With_ML_FLOW'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/rushikesh092002/Risk_Score_Predication_With_ML_FLOW.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "rushikesh092002"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "fcbbf35c568823de8126089cd6a25a106bd8afc4"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    mlflow_uri: str

In [7]:
from RiskScorePrediction.constants import *
from RiskScorePrediction.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.catboost
       
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            mlflow_uri="https://dagshub.com/rushikesh092002/Risk_Score_Predication_With_ML_FLOW.mlflow"
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from RiskScorePrediction.utils.common import save_json

In [10]:


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        target_col = 'RiskScore'
        test_x = test_data.drop([target_col], axis=1)
        test_y = test_data[[target_col]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme

       
        if mlflow.active_run():
            mlflow.end_run()

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

        
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

           
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="CatboostRegressor")
            else:
                mlflow.sklearn.log_model(model, "model")

       
        mlflow.end_run()


In [11]:
try:
    confif = ConfigurationManager()
    model_evaluation_config = confif.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-31 13:04:04,873: INFO: common: yaml file : config\config.yaml loaded  sucessfully]
[2025-01-31 13:04:04,878: INFO: common: yaml file : params.yaml loaded  sucessfully]
[2025-01-31 13:04:04,887: INFO: common: yaml file : schema.yaml loaded  sucessfully]
[2025-01-31 13:04:04,887: INFO: common: created directory at :artifacts]
[2025-01-31 13:04:04,897: INFO: common: created directory at :artifacts/model_evaluation]


[2025-01-31 13:04:07,474: INFO: common: json file saved at :artifacts\model_evaluation\metrics.json]


c:\Users\asus\anaconda3\envs\risk\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'CatboostRegressor'.
2025/01/31 13:04:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CatboostRegressor, version 1
Created version '1' of model 'CatboostRegressor'.
